<a href="https://colab.research.google.com/github/Birang1/opensource.guide/blob/main/2%EC%A3%BC%EC%B0%A8_yahoo_query_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# yahoo finance로 실시간 데이터 분석하기

# library 설치 하기
# 3가지 버전을 맞춰서 install 해야함.

!pip install urllib3==1.26.14 requests==2.27.0 yahooquery==2.3.0 

In [6]:
# 1. library import하기

from yahooquery import Ticker

# shift + Enter

In [7]:
company = Ticker('TSLA') # 2. 테슬라 불러오기 # TSLA : 종목코드

In [ ]:
company.summary_detail # 3. summary_detail 명령어 실행 : 테슬라에 관련된 summary 정보들

In [9]:
# get_modules 스니펫 : modules 라는 명령어에 필요한 정보들을 넣고
modules = 'summaryDetail assetProfile price financialData defaultKeyStatistics' 
all_data = company.get_modules(modules) # company에 입력해 놓은 회사의 필요한 정보들을 받는 코드인듯.

In [13]:
# 위에 정의한 코드 활용 ex . TSLA의 price 중에서 short name을 알고 싶다. # library의 설명대로 따라한것임. library의 설명서를 따라 하는것이 원래 생각보다 오래 걸림.
# 원하는 정보를 얻고 싶으면 https://yahooquery.dpguthrie.com/guide/ticker/modules/ < 이 페이지에서 원하는 항목을 친다음 어떠한 'module'에 속해 있는지 보면 된다.
# 하나하나의 정보를 알아 가는 방법

name = all_data['TSLA']['price']['shortName']
marketcap = all_data['TSLA']['price']['marketCap']
industry = all_data['TSLA']['assetProfile']['industry']
revenue = all_data['TSLA']['financialData']['totalRevenue']

print(name, marketcap, industry, revenue) # 테슬라, price 혹은 assetProfile에 해당하는 각 내용들을 보여줌. # 그런데 print()해서 오류가 자꾸 났었음.

Tesla, Inc. 602476314624 Auto Manufacturers 81462001664


In [ ]:
# 그 외 정보들 ex. balance_sheet이 가장 유명하고 거의 모든 정보들이 들어가 있음.

company.balance_sheet()
company.cash_flow()
company.earnings

In [48]:
# 원하는 정보들을 조합해서 잘 사용 할 수 있는가 (위의 내용들을 조합 하면 됨.) # 2주차에 궁극적으로 하려고 했던것 (build up 1)

# 1. library import하기

from yahooquery import Ticker

# shift + Enter

code = 'TSLA'
company = Ticker(code) # 2. 테슬라 불러오기 # TSLA : 종목코드

# get_modules 스니펫 : modules 라는 명령어에 필요한 정보들을 넣고

modules = 'summaryDetail assetProfile price financialData defaultKeyStatistics' # 필요한 정보들이 더 있다면 더 추가 해야 할듯.
all_data = company.get_modules(modules) # company에 입력해 놓은 회사의 필요한 정보들을 받는 코드인듯.

name = all_data[code]['price']['shortName']
marketcap = all_data[code]['price']['marketCap']
industry = all_data[code]['assetProfile']['industry']
summary = all_data[code]['assetProfile']['longBusinessSummary']

currentprice = all_data[code]['financialData']['currentPrice']
targetprice = all_data[code]['financialData']['targetMeanPrice']

per = all_data[code]['summaryDetail']['trailingPE']
eps = all_data[code]['defaultKeyStatistics']['trailingEps']
pbr = all_data[code]['defaultKeyStatistics']['priceToBook']

print(name, marketcap, industry, summary, currentprice, targetprice, per, eps, pbr)

Tesla, Inc. 602476314624 Auto Manufacturers Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. It operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; purchase financing and leasing services; services for electric vehicles through its company-owned service locations and Tesla mobile service technicians; and vehicle limited warranties and extended service plans. The Energy Generation and Storage segment engages in the design, manufacture, installation, sale, and leasing of solar energy generation

In [45]:
# 3년치 매출과 순이익을 보기 # 2주차에 궁극적으로 하려고 했던것 (build up 2)

company.earnings['TSLA'] # TSLA 까지 쳤을 때 나오는 정보에서 financial chart가 보고 싶다면,
company.earnings['TSLA']['financialsChart'] # financial chart
company.earnings['TSLA']['financialsChart']['yearly'] # 그중에서도 yearly를 보고 싶다면,

doc = company.earnings['TSLA']['financialsChart']['yearly'] # list가 됨.

# list는 보기 어려우니 dataframe으로 만들기
import pandas as pd
pd.DataFrame(doc)

df = pd.DataFrame(doc)

# dataframe 안의 x행의 y열을 보고 싶다면 df.iloc[x-1,y-1] 를 하면 됨.
df.iloc[0,0]

rev2019 = df.iloc[0,1] / 1000
rev2020 = df.iloc[1,1] / 1000
rev2021 = df.iloc[2,1] / 1000

ear2019 = df.iloc[0,2] / 1000
ear2020 = df.iloc[1,2] / 1000
ear2021 = df.iloc[2,2] / 1000

print(rev2019, rev2020, rev2021)

24578000.0 31536000.0 53823000.0


In [ ]:
# 2주차에서 하려고 했던것.

# 함수 만들어주기.


def add_company(code):
  # library import하기 : 이게 제일 먼저 우선으로   
  from yahooquery import Ticker
  import pandas as pd
  pd.options.display.float_format = '{:.2f}'.format # 소수점 둘째자리 까지 나오게 하는 코드
  company = Ticker(code) 
  
  # get_modules 스니펫 : modules 라는 명령어에 필요한 정보들을 넣고

  modules = 'summaryDetail assetProfile price financialData defaultKeyStatistics' # 필요한 정보들이 더 있다면 더 추가 해야 할듯.
  all_data = company.get_modules(modules) # company에 입력해 놓은 회사의 필요한 정보들을 받는 코드인듯.

  name = all_data[code]['price']['shortName']
  marketcap = all_data[code]['price']['marketCap']
  industry = all_data[code]['assetProfile']['industry']
  summary = all_data[code]['assetProfile']['longBusinessSummary']

  currentPrice = all_data[code]['financialData']['currentPrice']
  targetPrice = all_data[code]['financialData']['targetMeanPrice']

  per = all_data[code]['summaryDetail']['trailingPE']
  eps = all_data[code]['defaultKeyStatistics']['trailingEps']
  pbr = all_data[code]['defaultKeyStatistics']['priceToBook']

  ## 특정 회사의 특정 정보를 알수 있게 하는 code list를 짠것임.

  ###########################################################################

  # 3년치 매출과 순이익을 보기 & 이를 dataframe으로 만들어주기. # 2주차에 궁극적으로 하려고 했던것 (build up 2)

  doc = company.earnings[code]['financialsChart']['yearly'] # list가 됨.
  pd.DataFrame(doc)
  df = pd.DataFrame(doc)

  ###########################################################################

  # revenue랑 earning를 변수로 만들어 주기
  rev2019 = df.iloc[0,1] / 1000
  rev2020 = df.iloc[1,1] / 1000
  rev2021 = df.iloc[2,1] / 1000

  ear2019 = df.iloc[0,2] / 1000
  ear2020 = df.iloc[1,2] / 1000
  ear2021 = df.iloc[2,2] / 1000

  ###########################################################################

  # dictionary를 위에 만들어 놓은 변수로 지정

  doc = { # 위의 변수 이름과 똑같이 맞춰줘야 함.
      'code':code,
      'name':name,
      'industry':industry,
      'marketCap':marketcap/1000,
      'currentPrice':currentPrice,
      'targetPrice':targetPrice,
      'per':per,
      'eps':eps,
      'pbr':pbr,
      'rev2021':rev2021/1000,
      'rev2020':rev2020/1000,
      'rev2019':rev2019/1000,
      'ear2021':ear2021/1000,
      'ear2020':ear2020/1000,
      'ear2019':ear2019/1000,
  }

  return(doc)

add_company('TSLA')

In [ ]:
# 위에 만들어 놓은 add_company 함수를 가지고 진행.

df = pd.DataFrame() # data frame을 만드는데

codes = ['AAPL','ABNB','BIDU','META','GOOG','MSFT','TSLA','PYPL','NFLX','NVDA'] # 종목 코드를 list로 만들고

for code in codes: # 이 코드들을 반복하면서
  try:
    row = add_company(code) # 위에 만든 add_company를 위의 codes 안에 있는 회사 이름으로 계속 실행
    df = df.append(row, ignore_index = True) # 그것을 데이터 프레임으로 계속 붙혀 가는 작업을 진행
  except:
    print(f'error - {code}')

df

In [71]:
# 위에 만들어 놓은 df를 eps가 오름차순으로 만들어낼 수 있음.

df.sort_values(by='eps')

df.sort_values(by='eps', ascending = False) # 이렇게 하면 내림 차순이 됨.

# per가 30보다 작은 애들을 보기

cond = df['per'] < 30 
df[cond]
df[df['per'] < 30] # 이 상태가 dataframe이기 때문에

df[df['per'] < 30].sort_values(by='per', ascending=False) # 바로 .sort_values라는 함수를 보여주고 내림차순으로 보여주기

,code,name,industry,marketCap,currentPrice,targetPrice,per,eps,pbr,rev2021,rev2020,rev2019,ear2021,ear2020,ear2019
0,AAPL,Apple Inc.,Consumer Electronics,2.535459e+09,160.25,169.23,26.932774,5.95,44.750070,365817.0,274515.0,260174.0,94680.0,57411.0,55256.0
3,META,"Meta Platforms, Inc.",Internet Content & Information,5.341097e+08,206.01,219.71,23.043625,8.94,4.283665,117929.0,85965.0,70697.0,39370.0,29146.0,18485.0
4,GOOG,Alphabet Inc.,Internet Content & Information,1.354068e+09,106.06,124.78,22.470339,4.72,5.320291,257637.0,182527.0,161857.0,76033.0,40269.0,34343.0


In [ ]:
# 각 이름을 가진 column들을 모아 dataframe을 만듦.
df[['name','currentPrice','targetPrice']]

# 위의 dataframe을 변수로 만들어줌. *주의 : .copy()를 해줘야 함. 그렇지 않으면 나중에 변동이 일어남. 고정된 dataframe을 보여주고 싶다면 .copy()를 반드시 붙힐것.

new_df = df[['name','currentPrice','targetPrice']].copy()

# new_df에 gap 이라는 column을 새로 추가해주기 (각 행에 계산 된 값을 반복적으로 넣어주고 싶을 때.)

new_df['gap'] = new_df['targetPrice'] / new_df['currentPrice'] -1 

new_df

# 소수점이 많이 나올 때! 제공된 코드 스니펫 사용

pd.options.display.float_format = '{:.2f}'.format

new_df

new_df.sort_values(by='gap', ascending = False)

In [105]:
# 2주차 숙제

company = Ticker('TSLA')

df = company.balance_sheet()
df = df[['CashAndCashEquivalents']].transpose()

df.columns = ['2019','2020','2021','2022']

data = company.price
df['name'] = data['TSLA']['shortName']

df = df.reset_index(drop=True)

df[['name','2022','2021']]

new_df = df[['name','2022','2021']].copy()

new_df

new_df['diff'] = new_df['2022'] - new_df['2021']

new_df

,name,2022,2021,diff
0,"Tesla, Inc.",16253000000.00,17576000000.00,-1323000000.00
